Solves the following differential equation

  ```-d/dx (p(x) du/dx) + q(x)*u  =  f(x)```

by the Finite Element Method (FEM) using linear basis functions.

```u``` is an unknown function defined on ```[x0, x1]```, and ```p```, ```q``` and ```f``` are known functions of ```x```.

Essential boundary condition is applied at the left end.

Natural   boundary condition is applied at the right end.




---



In [ ]:
!pip install pycuda

Standard imports.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import sys

PyCUDA imports.

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
from pycuda.elementwise import ElementwiseKernel
from pycuda.compiler import SourceModule

```iDivUp``` function: if ```b``` divides ```a```, then ```a/b``` is returned, otherwise the function returns the integer division between ```a``` and ```b``` summed to ```1```.

In [ ]:
###################
# iDivUp FUNCTION #
###################
def iDivUp(a, b):
    # Round a / b to nearest higher integer value
    a = np.int32(a)
    b = np.int32(b)
    return (a / b + 1) if (a % b != 0) else (a / b)

Kernel functions.

In [ ]:
BLOCKSIZE = 256

mod = SourceModule("""

#include <stdio.h>

#define BLOCKSIZE		%(BBLOCKSIZE)s

#define PI_d			3.1415926535897932384626433832795028841971693993751058209749445923078164062

extern "C" {
/***************************************/
/* ATOMIC ADDITION FUNCTION ON DOUBLES */
/***************************************/
#if !defined(__CUDA_ARCH__) || __CUDA_ARCH__ >= 600
#else
__device__ double atomicAdd(double* address, double val)
{
	unsigned long long int* address_as_ull =
		(unsigned long long int*)address;
	register unsigned long long int old = *address_as_ull, assumed;
	do {
		assumed = old;
		old = atomicCAS(address_as_ull, assumed,
			__double_as_longlong(val +
			__longlong_as_double(assumed)));
	} while (assumed != old);
	return __longlong_as_double(old);
}
#endif

/**************************************/
/* ELEMENT CONNECTIVITY MATRIX KERNEL */
/**************************************/
__global__ void computeConnectivityMatrixKernel(int * __restrict__ d_elementConnectivityMatrix, const int numElements, const int numNodesPerElement)
{
	const int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if (tid >= numElements) return;

	d_elementConnectivityMatrix[tid * numNodesPerElement]				= tid;
	d_elementConnectivityMatrix[tid * numNodesPerElement + 1] 	= tid + 1;
	/* d_elementConnectivityMatrix[tid * numNodesPerElement * numElements]		= tid;
	d_elementConnectivityMatrix[tid * numNodesPerElement * numElements + 1] = tid + 1; */

}

/*********************************/
/* ASSEMBLE GLOBAL MATRIX KERNEL */
/*********************************/
__device__ void LinearBasisFunctions1D(const double xi, double * __restrict__ Ne1, double * __restrict__ Ne2, double * __restrict__ dNe1, double * __restrict__ dNe2) {

	/* --- Calculate the two basis functions at the natural coordinate xi */
	Ne1[0] = .5 * (1. - xi);
	Ne2[0] = .5 * (xi + 1.);

	/* --- Calculate the derivatives of the basis function with respect to natural coordinate xi */
	dNe1[0] = -.5;
	dNe2[0] =  .5;
}

__device__ double ff(double x) { return (16. * PI_d * PI_d + 1) * sin(4. * PI_d * x); }

__global__ void assembleGlobalMatrixKernel(const int * __restrict__ d_connectivityMatrix, const double * __restrict__ d_globalNodes, const double * __restrict__ d_gaussPoints, 
																		 			 const double * __restrict__ d_gaussWeights, double * __restrict__ d_f, int * __restrict__ d_I, int * __restrict__ d_J, double * __restrict__ d_X, 
									 									 			 const int numElements, const int numNodesPerElement, const int numberGaussPoints) {

	const int e = threadIdx.x + blockIdx.x * blockDim.x;

	if (e >= numElements) return;

	/* --- Remember:
	       1) Dynamically indexed arrays cannot be stored in registers, because the GPU register file is not dynamically addressable.
	       2) Scalar variables are automatically stored in registers by the compiler.
	       3) Statically - indexed (i.e.where the index can be determined at compile time), small arrays(say, less than 16 floats) may be stored in registers by the compiler. */
	double Ke[4] = {0., 0., 0., 0.};								/* --- Element stiffness matrix   */
	double fe[2] = {0., 0.};												/* --- Element force(load) vector */

	/* --- Global node numbers corresponding to the current element */
	int globalNodeNumbers[2];
	globalNodeNumbers[0] = d_connectivityMatrix[e * numNodesPerElement];
	globalNodeNumbers[1] = d_connectivityMatrix[e * numNodesPerElement + 1];

	/* --- Global coordinates of the element nodes as a column vector */
	double xe[2];
	xe[0] = d_globalNodes[globalNodeNumbers[0]];
	xe[1] = d_globalNodes[globalNodeNumbers[1]];

	double Ne1, Ne2, dNe1, dNe2, B1, B2;
	
	double x, Jacobian, JacxW;
	
	/* --- Calculate the element integral */
	for (int k = 0; k < numberGaussPoints; k++) {											/* --- Loop over all the Gauss points */

		/* --- Return the 1D linear basis functions along with their derivatives with respect to local coordinates xi at Gauss points */
		LinearBasisFunctions1D(d_gaussPoints[k], &Ne1, &Ne2, &dNe1, &dNe2);

		x = Ne1 * xe[0] + Ne2 * xe[1];                                  /* --- Global coordinate (here x) of the current integration point */

		Jacobian = dNe1 * xe[0] + dNe2 * xe[1];													/* --- Jacobian dx / dxi */

		JacxW = Jacobian * d_gaussWeights[k];														/* --- Calculate the integration weight */

		/* --- Calculate the derivatives of the basis functions with respect to x direction */
		B1 = dNe1 / Jacobian;                                
		B2 = dNe2 / Jacobian;                                

		Ke[0] = Ke[0] + (B1* B1) * JacxW;
		Ke[1] = Ke[1] + (B1* B2) * JacxW;
		Ke[2] = Ke[2] + (B2* B1) * JacxW;
		Ke[3] = Ke[3] + (B2* B2) * JacxW;

		fe[0] = fe[0] + ff(x) * Ne1 * JacxW;
		fe[1] = fe[1] + ff(x) * Ne2 * JacxW;

	}

	int globalIndexii, globalIndexjj;
	
	/* --- Loop over all the nodes of the e - th element */
	for (int ii = 0; ii < numNodesPerElement; ii++) {
		
		/* --- Global index of the ii - th local node of the e - th element */
		globalIndexii = d_connectivityMatrix[e * numNodesPerElement + ii];

		if (fe[ii] != 0) atomicAdd(&d_f[globalNodeNumbers[ii]], fe[ii]);		/* --- Assemble load */

		/* --- Loop over all the nodes of the e - th element */
		for (int jj = 0; jj < numNodesPerElement; jj++) {

			/* --- Global index of the ii - th local node of the e - th element */
			globalIndexjj = d_connectivityMatrix[e * numNodesPerElement + jj];

			/* --- If the element(ii, jj) of the stiffness matrix of the e - th element is different from zero, then add a triplet */
			if (Ke[ii * numNodesPerElement + jj] != 0) { 

				/* Main diagonal */
				if (globalIndexii == globalIndexjj) {
					d_I[3 * globalIndexii] = globalIndexii;
					d_J[3 * globalIndexii] = globalIndexjj;
					atomicAdd(&d_X[3 * globalIndexii], Ke[ii * numNodesPerElement + jj]);
				}
				/* Upper diagonal */
				else if (globalIndexii == globalIndexjj - 1) {
					d_I[3 * (globalIndexjj - 1) + 1] = globalIndexii;
					d_J[3 * (globalIndexjj - 1) + 1] = globalIndexjj;
					atomicAdd(&d_X[3 * (globalIndexjj - 1) + 1], Ke[ii * numNodesPerElement + jj]);
				}
				/* Lower diagonal */
				else if (globalIndexii == globalIndexjj + 1) {
					d_I[3 * (globalIndexii - 1) + 2] = globalIndexii;
					d_J[3 * (globalIndexii - 1) + 2] = globalIndexjj;
					atomicAdd(&d_X[3 * (globalIndexii - 1) + 2], Ke[ii * numNodesPerElement + jj]);
				}

			} 
		}
	
	}
}

__global__ void leftBoundaryConditionKernel(double * __restrict__ d_f, const int totalNumNodes) {

	double value = 4. * PI_d * cos(4. * PI_d);
	
	d_f[totalNumNodes - 1] = d_f[totalNumNodes - 1] + value;

}

__global__ void fixIndicesKernel(int * __restrict__ d_I, int * __restrict__ d_J, const int totalNumNodes) {

	const int tid = threadIdx.x + blockIdx.x * blockDim.x;
	
	if (tid >= totalNumNodes + 2 * (totalNumNodes - 1) - 3) return;
	
	d_I[tid] = d_I[tid] - 1;
	d_J[tid] = d_J[tid] - 1;

}

}
""" % { "BBLOCKSIZE" : BLOCKSIZE }, no_extern_c = True)

Set references to the ```__global__``` functions.

In [ ]:
computeConnectivityMatrix     = mod.get_function("computeConnectivityMatrixKernel")
d_linspace                    = ElementwiseKernel(
        "double *d_x, const double a, const double b, const int N",
        "d_x[i] = a + i * (b - a) / (double)(N - 1)")
assembleGlobalMatrix          = mod.get_function("assembleGlobalMatrixKernel")
leftBoundaryCondition         = mod.get_function("leftBoundaryConditionKernel")
fixIndices                    = mod.get_function("fixIndicesKernel")

Gaussian quadrature function.

In [ ]:
def gaussianQuadrature(ngp):

  gaussPoints       = np.zeros((ngp, 1))
  gaussWeights      = np.zeros((ngp, 1))
  
  # --- 1 Gauss point
  if (ngp == 1):       
    gaussPoints     = 0
    gaussWeights    = 2

  # --- 2 Gauss points
  elif (ngp == 2):   
    gaussPoints[0]  = - 0.577350269189625764509148780502
    gaussPoints[1]  =   0.577350269189625764509148780502

    gaussWeights[0] = 1.0
    gaussWeights[1] = 1.0

  # --- 3 Gauss points
  elif (ngp == 3):    

    gaussPoints[0]  = - 0.774596669241483377035853079956
    gaussPoints[1]  =   0.0;
    gaussPoints[2]  =   0.774596669241483377035853079956

    gaussWeights[0] = 5.0 / 9.0
    gaussWeights[1] = 8.0 / 9.0
    gaussWeights[2] = 5.0 / 9.0
    
  # --- 4 Gauss points
  elif (ngp == 4): 

    gaussPoints[0]  = - 0.861136311594052575223946488893
    gaussPoints[1]  = - 0.339981043584856264802665759103
    gaussPoints[2]  =   0.339981043584856264802665759103
    gaussPoints[3]  =   0.861136311594052575223946488893

    gaussWeights[0] = 0.347854845137453857373063949222
    gaussWeights[1] = 0.652145154862546142626936050778
    gaussWeights[2] = 0.652145154862546142626936050778
    gaussWeights[3] = 0.347854845137453857373063949222

  # --- 5 Gauss points
  elif (ngp == 5): 

    gaussPoints[0]  = - 0.906179845938663992797626878299
    gaussPoints[1]  = - 0.538469310105683091036314420700
    gaussPoints[2]  =   0.0
    gaussPoints[3]  =   0.538469310105683091036314420700
    gaussPoints[4]  =   0.906179845938663992797626878299

    gaussWeights[0] = 0.236926885056189087514264040720
    gaussWeights[1] = 0.478628670499366468041291514836
    gaussWeights[2] = 0.568888888888888888888888888889
    gaussWeights[3] = 0.478628670499366468041291514836
    gaussWeights[4] = 0.236926885056189087514264040720

  else:

    print("\nGaussian quadrature - Fatal error! \n")
    print("Illegal number of Gauss points = ", ngp, "\n")
    print("Legal values are 1 to 5.\n")
    sys.exit()

  return gaussPoints, gaussWeights

Parameters

In [ ]:
x0                      = 0.0                                   # --- Left end of the 1D domain
x1                      = 1.0                                   # --- Right end of the 1D domain

numElements             = 10                                    # --- Total number of elements

totalNumNodes           = numElements + 1                       # --- Total number of nodes
numNodesPerElement      = 2                                     # --- Number of nodes per element
     
numberGaussPoints       = 2                                     # --- Number of Gauss points

Nodes and connectivity matrix.

In [ ]:
# --- Global node number = connectivityMatrix(local node number, element number)
# --- d_elementConnectivityMatrix is a (numElements x numNodesPerElement) matrix, stored rowwise
d_elementConnectivityMatrix = gpuarray.zeros((numElements, numNodesPerElement), dtype = np.int32)

d_globalNodes               = gpuarray.zeros((totalNumNodes, ), dtype = np.float64)

blockDimConnectivity  = (BLOCKSIZE, 1, 1)
gridDimConnectivity   = (int(iDivUp(numElements, BLOCKSIZE)), 1, 1)

computeConnectivityMatrix(d_elementConnectivityMatrix, np.int32(numElements), np.int32(numNodesPerElement), block = blockDimConnectivity, grid = gridDimConnectivity)

d_linspace(d_globalNodes, x0, x1, totalNumNodes)

Definition of global force and solution vectors.

In [ ]:
# --- Global force vector
d_f               = gpuarray.zeros((totalNumNodes, ), dtype = np.float64)
# --- Global solution vector
d_d               = gpuarray.zeros((totalNumNodes, ), dtype = np.float64)

Definition of (I, J, K) triplet for assembling the sparse matrix.

In [ ]:
# --- Row indices of non - zero entries
d_I               = gpuarray.zeros((totalNumNodes + 2 * (totalNumNodes - 1), ), dtype = np.int32)
# --- Column indices of non - zero entries
d_J               = gpuarray.zeros((totalNumNodes + 2 * (totalNumNodes - 1), ), dtype = np.int32)
# --- Non - zero entries matrix	
d_X               = gpuarray.zeros((totalNumNodes + 2 * (totalNumNodes - 1), ), dtype = np.float64)

Compute Gaussian quadrature points.

In [ ]:
gaussPoints, gaussWeights = gaussianQuadrature(numberGaussPoints)     # --- Return Gauss quadrature points and weights
  
d_gaussPoints   = gpuarray.to_gpu(gaussPoints)
d_gaussWeights  = gpuarray.to_gpu(gaussWeights)

Assemble global stiffness matrix.

In [ ]:
assembleGlobalMatrix(d_elementConnectivityMatrix, d_globalNodes, d_gaussPoints, d_gaussWeights, 
		d_f, d_I, d_J, d_X, np.int32(numElements), np.int32(numNodesPerElement), np.int32(numberGaussPoints), block = blockDimConnectivity, grid = gridDimConnectivity)

Sorting.

In [ ]:
d_keys = d_I * totalNumNodes + d_J

keys = d_keys.get()
print(keys)

[  0   1  11  12  13  23  24  25  35  36  37  47  48  49  59  60  61  71
  72  73  83  84  85  95  96  97 107 108 109 119 120]


In [ ]:
f = d_f.get()

d_I = d_I[3 : totalNumNodes + 2 * (totalNumNodes - 1)]
d_J = d_J[3 : totalNumNodes + 2 * (totalNumNodes - 1)]
d_X = d_X[3 : totalNumNodes + 2 * (totalNumNodes - 1)]

blockDimIndices  = (BLOCKSIZE, 1, 1)
gridDimIndices   = (int(iDivUp(totalNumNodes + 2 * (totalNumNodes - 1) - 3, BLOCKSIZE)), 1, 1)
fixIndices(d_I, d_J, np.int32(totalNumNodes), block = blockDimIndices, grid = gridDimIndices)

I = d_I.get()
J = d_J.get()
X = d_X.get()

print(I)
print(J)
print(X)

print(f)

[0 0 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 7 7 7 8 8 8 9 9]
[0 1 0 1 2 1 2 3 2 3 4 3 4 5 4 5 6 5 6 7 6 7 8 7 8 9 8 9]
[ 20. -10. -10.  20. -10. -10.  20. -10. -10.  20. -10. -10.  20. -10.
 -10.  20. -10. -10.  20. -10. -10.  20. -10. -10.  20. -10. -10.  10.]
[ 3.04943043e+00  1.32514454e+01  8.18984366e+00 -8.18984366e+00
 -1.32514454e+01 -3.10862447e-15  1.32514454e+01  8.18984366e+00
 -8.18984366e+00 -1.32514454e+01 -3.04943043e+00]


In [ ]:
import ctypes

# --- Loading cuSparse library
cuSparse = ctypes.cdll.LoadLibrary('libcusparse.so')

# --- Function interfaces
# handle    -> ctypes.c_void_p
# int       -> ctypes.c_int
# double    -> ctypes.c_double
# int *     -> ctypes.c_void_p
# double *  -> ctypes.c_void_p
#     cusparseCreate
cuSparse.cusparseCreate.restype = int
cuSparse.cusparseCreate.argtypes = [ctypes.c_void_p]
#     cusparseGetVersion
cuSparse.cusparseGetVersion.restype = int
cuSparse.cusparseGetVersion.argtypes = [ctypes.c_int, ctypes.c_void_p]
#     cusparseXcoo2csr
cuSparse.cusparseXcoo2csr.restype = int
cuSparse.cusparseXcoo2csr.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_int, ctypes.c_int, ctypes.c_void_p, ctypes.c_int]
#     cusparseCreateMatDescr
cuSparse.cusparseCreateMatDescr.restype = int
cuSparse.cusparseCreateMatDescr.argtypes = [ctypes.c_void_p]
#     cusparseDcsr2dense
cuSparse.cusparseDcsr2dense.restype = int
cuSparse.cusparseDcsr2dense.argtypes = [ctypes.c_void_p, ctypes.c_int, ctypes.c_int, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_int]

# --- Create cuSPARSE handle
cuSparseHandlePointer = ctypes.c_void_p()
status = cuSparse.cusparseCreate(ctypes.byref(cuSparseHandlePointer))
cuSparseHandle = cuSparseHandlePointer.value
print("cusparseCreate status: ", status)

# --- Return cuSparse version
cuSparseVersionPointer = ctypes.c_void_p()
status = cuSparse.cusparseGetVersion(cuSparseHandle, ctypes.byref(cuSparseVersionPointer))
print("cusparseGetVersion status: ", status)
print("cuSparse version: ", cuSparseVersionPointer.value)

# --- Convert from coo to csr
# cusparseStatus_t cusparseXcoo2csr(cusparseHandle_t handle, const int *cooRowInd, int nnz, int m, int *csrRowPtr, cusparseIndexBase_t idxBase)
d_csrRowPtr          = gpuarray.zeros((totalNumNodes, ), dtype = np.int32)
status = cuSparse.cusparseXcoo2csr(cuSparseHandle, int(d_I.gpudata), totalNumNodes + 2 * (totalNumNodes - 1) - 3, totalNumNodes - 1, int(d_csrRowPtr.gpudata), 0)
print("cusparseXcoo2csr status: ", status)
csrRowPtr = d_csrRowPtr.get()
print(csrRowPtr)

# --- Create matrix descriptor
descrA = ctypes.c_void_p()
status = cuSparse.cusparseCreateMatDescr(ctypes.byref(descrA))
print("cusparseCreateMatDescr status: ", status)

# --- Convert from csr to dense
d_A          = gpuarray.zeros((totalNumNodes - 1, totalNumNodes - 1), dtype = np.float64)
# cusparseStatus_t cusparseDcsr2dense(cusparseHandle_t handle, int m, int n, const cusparseMatDescr_t descrA, const double *csrValA, const int *csrRowPtrA, const int *csrColIndA, double *A, int lda)
status = cuSparse.cusparseDcsr2dense(cuSparseHandle, totalNumNodes - 1, totalNumNodes - 1, descrA, int(d_X.gpudata), int(d_csrRowPtr.gpudata), int(d_J.gpudata), int(d_A.gpudata), totalNumNodes - 1)
print("cusparseDcsr2dense status: ", status)
A = d_A.get()
print(A)

cusparseCreate status:  0
cusparseGetVersion status:  0
cuSparse version:  10000
cusparseXcoo2csr status:  0
[ 0  2  5  8 11 14 17 20 23 26 28]
cusparseCreateMatDescr status:  0
cusparseDcsr2dense status:  0
[[ 20. -10.   0.   0.   0.   0.   0.   0.   0.   0.]
 [-10.  20. -10.   0.   0.   0.   0.   0.   0.   0.]
 [  0. -10.  20. -10.   0.   0.   0.   0.   0.   0.]
 [  0.   0. -10.  20. -10.   0.   0.   0.   0.   0.]
 [  0.   0.   0. -10.  20. -10.   0.   0.   0.   0.]
 [  0.   0.   0.   0. -10.  20. -10.   0.   0.   0.]
 [  0.   0.   0.   0.   0. -10.  20. -10.   0.   0.]
 [  0.   0.   0.   0.   0.   0. -10.  20. -10.   0.]
 [  0.   0.   0.   0.   0.   0.   0. -10.  20. -10.]
 [  0.   0.   0.   0.   0.   0.   0.   0. -10.  10.]]


Letf boundary condition.

In [ ]:
leftBoundaryCondition(d_f, np.int32(totalNumNodes), block = (1, 1, 1), grid = (1, 1, 1))

d_f = d_f[1 : totalNumNodes]
f = d_f.get()
print(f)

[ 1.32514454e+01  8.18984366e+00 -8.18984366e+00 -1.32514454e+01
 -3.10862447e-15  1.32514454e+01  8.18984366e+00 -8.18984366e+00
 -1.32514454e+01  9.51694018e+00]


Solve the linear system.

In [ ]:
cuSolver = ctypes.cdll.LoadLibrary('libcusolver.so')

cuSolver.cusolverSpCreate.restype = int
cuSolver.cusolverSpCreate.argtypes = [ctypes.c_void_p]

cuSolverHandle = ctypes.c_void_p()
status = cuSolver.cusolverSpCreate(ctypes.byref(cuSolverHandle))

cuSolver.cusolverSpDcsrlsvqr.restype = int
cuSolver.cusolverSpDcsrlsvqr.argtypes= [ctypes.c_void_p, ctypes.c_int, ctypes.c_int, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_double, ctypes.c_int, ctypes.c_void_p, ctypes.c_void_p]

d_d          = gpuarray.zeros((totalNumNodes - 1,), dtype = np.float64)
tol          = ctypes.c_double(1e-10)
reorder      = ctypes.c_int(0)
singularity  = ctypes.c_int(99)
cuSolver.cusolverSpDcsrlsvqr(cuSolverHandle, totalNumNodes - 1, totalNumNodes + 2 * (totalNumNodes - 1) - 3, descrA, int(d_X.gpudata), int(d_csrRowPtr.gpudata), int(d_J.gpudata), int(d_f.gpudata), tol, reorder, int(d_d.gpudata), ctypes.byref(singularity))

0

In [ ]:
d = d_d.get()

print(d)

[ 0.95169402  0.5782435  -0.61419139 -0.98764192 -0.0359479   0.91574612
  0.5422956  -0.65013929 -1.02358981 -0.07189579]


https://stackoverflow.com/questions/30460074/interfacing-cusolver-sparse-using-pycuda

https://scikit-cuda.readthedocs.io/en/latest/genindex.html

https://github.com/lebedov/scikit-cuda/blob/master/skcuda/cusparse.py

https://docs.nvidia.com/cuda/cusparse/index.html